In [1]:
// import edu.umd.cs.hcil.spark.analytics.utils.JsonUtils
// import edu.umd.cs.hcil.spark.analytics.utils.StatusTokenizer
// import edu.umd.cs.hcil.spark.analytics.TweetsToGraph

import edu.umd.cs.hcil.models.twitter.TweetParser
import edu.umd.cs.hcil.analytics.spark.network.twitter.RetweetGraph
import edu.umd.cs.hcil.analytics.spark.network.community.LouvainCommunity

In [2]:
import org.apache.spark.graphx._

In [3]:
import java.io.{File,BufferedWriter,FileWriter}

In [4]:
val tweets = sc.textFile("hdfs://dumbo/user/clb617/wri/data/stream_0{1,3,4}/statuses.*.gz")
// val tweets = sc.textFile("file:/scratch/clb617/data/wri/wri_2018/data/stream_0{1,3,4}/statuses.*.gz")
// tweets.persist(org.apache.spark.storage.StorageLevel.MEMORY_AND_DISK_SER)
// val tweet_count = tweets.count()

// print("Tweet Count: " + tweet_count)

Waiting for a Spark session to start...

tweets = hdfs://dumbo/user/clb617/wri/data/stream_0{1,3,4}/statuses.*.gz MapPartitionsRDD[1] at textFile at <console>:34


hdfs://dumbo/user/clb617/wri/data/stream_0{1,3,4}/statuses.*.gz MapPartitionsRDD[1] at textFile at <console>:34

In [5]:
val tweet_objs = tweets.map(tjson => TweetParser.parseJson(tjson))
    .filter(t => t != null)
    .map(t => (t.getId, t))
    .reduceByKey((l, r) => l)
    .map(tup => tup._2)
    .cache()

val dedup_count = tweet_objs.count()
print("Dedpulicated Tweet Count: " + dedup_count)

Dedpulicated Tweet Count: 812366                                                

tweet_objs = MapPartitionsRDD[6] at map at <console>:40
dedup_count = 812366


812366

In [6]:
val tweet_objs_clean = tweet_objs//.
//     filter(t => StatusTokenizer.tokenize(t).length > 3 && 
//                                          t.getHashtagEntities.length < 3 &&
//                                          t.getURLEntities.length < 3)

tweet_objs_clean = MapPartitionsRDD[6] at map at <console>:40


MapPartitionsRDD[6] at map at <console>:40

In [7]:
val orig_graph = RetweetGraph.getGraph(tweet_objs_clean, 2)
val graph = orig_graph.partitionBy(org.apache.spark.graphx.PartitionStrategy.RandomVertexCut, 64)
graph.cache()

orig_graph = org.apache.spark.graphx.impl.GraphImpl@3c893407
graph = org.apache.spark.graphx.impl.GraphImpl@3ed22de0


org.apache.spark.graphx.impl.GraphImpl@3ed22de0

In [8]:
val comm_graph = org.apache.spark.graphx.lib.LabelPropagation.run(graph, 30)

[Stage 1768:======================================>         (1438 + 122) / 1809]]

comm_graph = org.apache.spark.graphx.impl.GraphImpl@4781c61


org.apache.spark.graphx.impl.GraphImpl@4781c61

In [9]:
val unique_labels = comm_graph.vertices.map(pair => pair._2).distinct()
val label_count = unique_labels.count()
println("Number of Communities: " + label_count)

Number of Communities: 2531                                                     


unique_labels = MapPartitionsRDD[588] at distinct at <console>:45
label_count = 2531


2531

In [10]:
val topCommCount = 100

val comm_counts = comm_graph.vertices.map(v => (v._2, 1)).reduceByKey((l, r) => l + r).collect()
val ordered_comm = comm_counts.sortBy(pair => pair._2).reverse.take(topCommCount)

println("Community Counts:")
for ( commPair <- ordered_comm ) {
    println("\t" + commPair)
}

Community Counts:                                                               
	(40955185,11670)
	(40001204,5583)
	(723699160604655616,4435)
	(110260249,2541)
	(122448174,1713)
	(2891380160,1218)
	(2435623728,827)
	(904910426470621185,588)
	(995999075030982656,563)
	(51798186,400)
	(786764,244)
	(3690282255,222)
	(80047141,157)
	(729310106408595456,147)
	(471271042,139)
	(3400553939,98)
	(36711022,96)
	(987628196135776256,95)
	(628453349,91)
	(265468729,83)
	(312182362,80)
	(533317016,63)
	(885159387089612808,58)
	(855058258482089984,56)
	(982312407283458048,55)
	(3056091634,50)
	(132945441,48)
	(935034454778793984,43)
	(127483019,43)
	(297383888,41)
	(797154838610059264,40)
	(1009733536390057984,38)
	(832409029439868932,35)
	(47401488,33)
	(972986153208111104,29)
	(792321770011422720,29)
	(2521698572,28)
	(1227253801,28)
	(390981850,28)
	(2610727407,28)
	(6761882,27)
	(957048252385366016,25)
	(2428862994,24)
	(1026668848257351681,23)
	(48462491,23)
	(1097805626,21)
	(4820804277,21)


topCommCount = 100
comm_counts = Array((1641649411,3), (181949222,1), (942246662914764800,20), (1635648960,3), (38678233,1), (953979235500265472,1), (224757400,1), (150056555,1), (2509996248,1), (179038544,1), (931924417197887489,1), (2463052700,1), (174593833,1), (2149285269,1), (704002464249462786,1), (874782241473220608,1), (807305067976073216,1), (988147538728574976,1), (42104488,1), (4901597370,1), (719430270,1), (3176279903,1), (839468493082923010,11), (982322777385349120,9), (203725981,1), (2557410152,1), (61126132,1), (2279627352,1), (584482497,1), (424134547,1), (3587244915,1), (1317132926,1), (4091449396,1), (174161502,3), (393812093,1), (34063500,1), (468603391,1), (1009752403552362497,1), (724647699950239744,4), (192510346...


[(1641649411,3), (181949222,1), (942246662914764800,20), (1635648960,3), (38678233,1), (953979235500265472,1), (224757400,1), (150056555,1), (2509996248,1), (179038544,1), (931924417197887489,1), (2463052700,1), (174593833,1), (2149285269,1), (704002464249462786,1), (874782241473220608,1), (807305067976073216,1), (988147538728574976,1), (42104488,1), (4901597370,1), (719430270,1), (3176279903,1), (839468493082923010,11), (982322777385349120,9), (203725981,1), (2557410152,1), (61126132,1), (2279627352,1), (584482497,1), (424134547,1), (3587244915,1), (1317132926,1), (4091449396,1), (174161502,3), (393812093,1), (34063500,1), (468603391,1), (1009752403552362497,1), (724647699950239744,4), (1925103463,1), (823751702,1), (363614463,1), (2693740027,1), (968047298516103169,3), (4727492000,1), (738094177137762305,1), (65427953,1), (751438742,1), (98362607,1), (485931813,3), (3072835883,2), (984218536158679040,1), (455982012,1), (708319387875483648,1), (70355674,1), (1026225847,1), (20944649,1

In [11]:
val filtered_comms = ordered_comm.filter(tup => tup._2 > 100)

filtered_comms = Array((40955185,11670), (40001204,5583), (723699160604655616,4435), (110260249,2541), (122448174,1713), (2891380160,1218), (2435623728,827), (904910426470621185,588), (995999075030982656,563), (51798186,400), (786764,244), (3690282255,222), (80047141,157), (729310106408595456,147), (471271042,139))


[(40955185,11670), (40001204,5583), (723699160604655616,4435), (110260249,2541), (122448174,1713), (2891380160,1218), (2435623728,827), (904910426470621185,588), (995999075030982656,563), (51798186,400), (786764,244), (3690282255,222), (80047141,157), (729310106408595456,147), (471271042,139)]

In [12]:
val file = new File("wri-retweet-community-2018.log")
val bw = new BufferedWriter(new FileWriter(file))

def localLog(msg : String) : Unit = {
    kernel.out.println(msg)
    bw.write(msg + "\n")
    bw.flush
}

file = wri-retweet-community-2018.log
bw = java.io.BufferedWriter@3d9a43d1


localLog: (msg: String)Unit


java.io.BufferedWriter@3d9a43d1

In [13]:
val tolerance = 0.01

var resultMap = List[(String,Any)]()
var userCommunityMap = Map[Long,Array[Long]]()

for ( commPair <- filtered_comms ) {
    val commId = commPair._1
    val commSize = commPair._2
    
    localLog("Community ID: " + commId)
    
    val subverts = comm_graph.vertices.filter(p => p._2 == commId).map(p => p._1).collect()
    
    val comms = graph.subgraph(vpred = (id, attr) => subverts.contains(id))

    // Create a list of all users with their community assignments
    userCommunityMap = userCommunityMap ++ Map(commId -> subverts)
    
//     val commSizeV2 = comms.vertices.count()
    
//     localLog("\t" + commSize + ", " + commSizeV2)
    
//     val rankedVertices = comms.pageRank(tolerance).vertices
//     val rankedGraph = comms.outerJoinVertices(rankedVertices) { (id, user, rankOpt) =>
//       rankOpt match {
//         case Some(rank) => (user, rank)
//         case None => (user, 0d) // No degreeOpt means zero degree
//       }
//     }

//     val topAccts = rankedGraph.vertices.sortBy(tuple => tuple._2._2, false).take(10)
//     resultMap = resultMap :+ (commId.toString, topAccts)
    
//     for (x <- topAccts) {
//         localLog("\t" + "%d, %s, %f".format(x._1, x._2._1.name, x._2._2))
//     }
}

Community ID: 40955185
Community ID: 40001204                                                          
Community ID: 723699160604655616                                                
Community ID: 110260249                                                         
[Stage 2364:=============================================>   (1676 + 81) / 1809]Community ID: 122448174
Community ID: 2891380160                                                        
Community ID: 2435623728                                                        
[Stage 2661:==============================================>  (1715 + 61) / 1809]Community ID: 904910426470621185
[Stage 2760:============================================>    (1642 + 87) / 1809]Community ID: 995999075030982656
[Stage 2859:===============================================> (1763 + 36) / 1809]Community ID: 51798186
[Stage 2958:==============================================>  (1702 + 70) / 1809]Community ID: 786764
Community ID: 3690282255              

tolerance = 0.01
resultMap = List()
userCommunityMap = Map(723699160604655616 -> Array(365021829, 1411917264, 3970723945, 810839988208340993, 845526601, 1031826172882116614, 273784915, 81728812, 557287777, 2789825028, 846484290049236992, 778442302746337280, 114117150, 309378801, 1017252372232527872, 18728580, 701452402466029568, 170460265, 93355258, 121620884, 923568904454025217, 870823826401050624, 431777552, 96723618, 1597451928, 554056909, 801507974422228992, 1021812158617845761, 953417865834778625, 893524523055796225, 936298549318504448, 560764683, 1902476468, 322993344, 90629104, 83782039, 331748905, 996589866367508480, 114887795, 1037011569031311361, 760223220, 585254931, 399892128, 181037500, 166650523...


Map(723699160604655616 -> [J@978cdd8, 2435623728 -> [J@691138f6, 995999075030982656 -> [J@5d267a2c, 904910426470621185 -> [J@528bc0f8, 471271042 -> [J@1aeb815b, 3690282255 -> [J@b02f4e0, 729310106408595456 -> [J@155ae324, 51798186 -> [J@589d0165, 40001204 -> [J@4f841d1f, 2891380160 -> [J@38bf3bae, 110260249 -> [J@1ca297cf, 40955185 -> [J@5d5591ba, 122448174 -> [J@3ca28b1b, 786764 -> [J@e2b7b46, 80047141 -> [J@302943bb)

In [14]:
bw.close()

In [15]:
userCommunityMap.keys

Set(723699160604655616, 2435623728, 995999075030982656, 904910426470621185, 471271042, 3690282255, 729310106408595456, 51798186, 40001204, 2891380160, 110260249, 40955185, 122448174, 786764, 80047141)

In [16]:
val file = new File("communities_2018.csv")
val bw = new BufferedWriter(new FileWriter(file))

bw.write("user,community\n")
for ( k <- userCommunityMap.keys ) {
    for ( u <- userCommunityMap(k) ) {
        bw.write("%d,%d\n".format(u,k))
    }
}

bw.close()

file = communities_2018.csv
bw = java.io.BufferedWriter@3a3e2fa2


java.io.BufferedWriter@3a3e2fa2

In [17]:
print(userCommunityMap.keys)

Set(723699160604655616, 2435623728, 995999075030982656, 904910426470621185, 471271042, 3690282255, 729310106408595456, 51798186, 40001204, 2891380160, 110260249, 40955185, 122448174, 786764, 80047141)